In [1]:
import pandas as pd
import dataframe_image as dfi
import requests
import os
import json
import requests
import warnings
from bs4 import BeautifulSoup
os.makedirs("./Result",exist_ok=True)



# 國內成分證券ETF
https://www.twse.com.tw/zh/products/securities/etf/products/domestic.html

In [2]:


url='https://www.twse.com.tw/rwd/zh/ETF/domestic?response=json&_=1681734086399'

headers = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36"
}
req = requests.get(url, headers = headers,verify=False)
ETFdata = req.content.decode('utf8')
ETFdf=pd.DataFrame(json.loads(ETFdata)['data'])
ETFdf=ETFdf.dropna()
ETFdf.columns = ['證券代號', '證券簡稱','']

C:\Users\user\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.twse.com.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(



# ETF   持股 Data

In [3]:
# https://www.moneydj.com/ETF/X/Basic/Basic0007A.xdjhtm?etfid=0050.TW
# https://www.moneydj.com/etf/eb/et305001list.djhtm?order=4&c=47
def get_erfBaseData(etfid,rtfdf):
    etfurl='https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid='+etfid+'.TW'
    etfresp = requests.get(etfurl)
    etfresp.encoding = 'utf-8'    #轉換編碼至UTF-8
    print("utl="+etfurl)
    if etfresp.status_code==200 :
        etfsoup = BeautifulSoup(etfresp.text, 'lxml')
        
        #印出表格中的所有文字內容
        for tr in etfsoup.find('table', id="Repeater1").find_all('tr'):
            td=tr.find_all('td')
            row = [i.text for i in td]
            row.append(etfid)
            if len(row)>0:
                rtfdf = rtfdf.append([row],ignore_index=True)
        for tr in etfsoup.find('table', id="Repeater2").find_all('tr'):
            td=tr.find_all('td')
            row = [i.text for i in td]
            if len(row)>0:
                row.append(etfid)
                rtfdf = rtfdf.append([row],ignore_index=True)

        rtfdf=rtfdf.dropna()
       
        return rtfdf

In [4]:
rtfdf = pd.DataFrame()
for  etfid in ETFdf['證券代號'].tolist():
    rtfdf=get_erfBaseData(etfid,rtfdf)
# 【Output】   
rtfdf.columns = ['股票名稱', '持股(千股)','比例','增減','ETF_Id']
rtfdf.to_csv(r'BaseInfo/ETFBaseData.csv', index = False, encoding='utf-8', errors='ignore')
rtfdf.to_csv(r'Result/ETFBaseDataRead.csv', index = False,encoding='big5', errors='ignore')

utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0050.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0051.TW


C:\Users\user\AppData\Local\Temp\ipykernel_9948\1022840050.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rtfdf = rtfdf.append([row],ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_9948\1022840050.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rtfdf = rtfdf.append([row],ignore_index=True)


utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0052.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0053.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0055.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0056.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0057.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=006203.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=006204.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=006208.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=00690.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=00692.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=00701.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=00713.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=00730.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xd

In [5]:



StockBaseData=pd.read_csv('BaseInfo/StockBaseData.csv')
ETFBaseData=pd.read_csv('BaseInfo/ETFBaseData.csv')

In [6]:
# 兩表Join
df=pd.merge(ETFBaseData, StockBaseData, how='left', left_on='股票名稱', right_on='公司簡稱')
# 處理格式問題
df['ETF_Id']='00'+df['ETF_Id'].astype(str)

#處理上市公司產業類別對應不到的狀況
df['上市公司產業類別']=df['上市公司產業類別'].fillna('未分類')
print(df.columns)

Index(['股票名稱', '持股(千股)', '比例', '增減', 'ETF_Id', '公司代號', '公司名稱', '公司簡稱', '產業別',
       '實收資本額', 'Type', '上市公司產業類別', '上櫃公司產業類別', '新編碼'],
      dtype='object')


In [7]:
# 取得 etf 對應持股佔比
df['data']=df.agg(lambda x: f"{x['股票名稱']} ( {x['比例']}%)", axis=1)
df.sort_values('比例', ascending=False)
t=df[['ETF_Id','data']]
t[t['ETF_Id']=='00713']

,ETF_Id,data
1099,00713,台灣大 ( 9.76%)
1100,00713,統一 ( 9.29%)
1101,00713,遠傳 ( 9.0%)
1102,00713,統一超 ( 5.74%)
1103,00713,台新金 ( 3.57%)
1104,00713,華南金 ( 3.54%)
1105,00713,中信金 ( 3.12%)
1106,00713,聯發科 ( 3.03%)
1107,00713,玉山金 ( 2.72%)
1108,00713,宏全 ( 2.12%)


In [8]:
keydata=df.groupby(['ETF_Id','上市公司產業類別'])['data'].apply(','.join).reset_index()
keydata

,ETF_Id,上市公司產業類別,data
0,0050,光電業,大立光 ( 0.56%)
1,0050,其他,中租-KY ( 0.44%)
2,0050,其他電子業,鴻海 ( 4.4%)
3,0050,半導體業,"台積電 ( 55.49%),聯發科 ( 5.24%),聯電 ( 1.33%),日月光投控 (..."
4,0050,塑膠工業,"南亞 ( 0.39%),台塑 ( 0.35%)"
...,...,...,...
781,009803,運動休閒,"寶成 ( 1.28%),復盛應用 ( 0.59%)"
782,009803,金融保險,"富邦金 ( 5.45%),國泰金 ( 3.38%),中信金 ( 2.24%),兆豐金 ( 1..."
783,009803,電子通路業,"崇越 ( 0.64%),文曄 ( 0.01%),聯強 ( 0.01%)"
784,009803,電子零組件業,"台達電 ( 3.48%),川湖 ( 1.36%),台光電 ( 0.57%),嘉澤 ( 0.3..."


In [9]:
# 取得產業比例排名 by ETF_Id 上市公司產業類別

# 挑選要用的欄位
colList=['ETF_Id','股票名稱', '上市公司產業類別','持股(千股)','比例']
df_=df[colList]
# groupby
g=df_.groupby(['ETF_Id','上市公司產業類別']).sum().reset_index()
# 排序 取前三
#g.sort_values('比例', ascending=False).groupby(['ETF_Id']).head(3).sort_values(['ETF_Id', '比例'], ascending=[True, False])
# 排序(這個對未來比較有彈性)
g['row_numbers'] =g.groupby('ETF_Id')['比例'].rank(ascending=False,method='dense')
g[g['ETF_Id']=='00713']

C:\Users\user\AppData\Local\Temp\ipykernel_9948\2400789445.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  g=df_.groupby(['ETF_Id','上市公司產業類別']).sum().reset_index()


,ETF_Id,上市公司產業類別,比例,row_numbers
226,00713,光電業,3.11,9.0
227,00713,其他,3.93,8.0
228,00713,其他電子業,4.20,7.0
229,00713,化學工業,1.13,17.0
230,00713,半導體業,7.58,4.0
231,00713,建材營造,0.98,18.0
232,00713,汽車工業,1.33,15.0
233,00713,紡織纖維,1.24,16.0
234,00713,航運業,2.59,11.0
235,00713,貿易百貨,5.74,5.0


In [10]:
#轉成橫的方便看
df1=g[g['row_numbers']==1].merge(keydata,on=['ETF_Id','上市公司產業類別'])
df2=g[g['row_numbers']==2].merge(keydata,on=['ETF_Id','上市公司產業類別'])
df3=g[g['row_numbers']==3].merge(keydata,on=['ETF_Id','上市公司產業類別'])
df_result=df1.merge(df2,on='ETF_Id',suffixes=["","_2"]).merge(df3,on='ETF_Id',suffixes=["","_3"])
df_result = df_result.drop(df_result.filter(regex='row_numbers').columns, axis=1)
df_result

,ETF_Id,上市公司產業類別,比例,data,上市公司產業類別_2,比例_2,data_2,上市公司產業類別_3,比例_3,data_3
0,0050,半導體業,65.18,"台積電 ( 55.49%),聯發科 ( 5.24%),聯電 ( 1.33%),日月光投控 (...",金融保險,12.31,"富邦金 ( 1.75%),中信金 ( 1.63%),國泰金 ( 1.42%),兆豐金 ( 1...",電腦及週邊設備業,5.45,"廣達 ( 1.52%),華碩 ( 1.1%),緯創 ( 0.66%),光寶科 ( 0.49%..."
1,0051,電子零組件業,13.45,"台光電 ( 2.4%),川湖 ( 1.49%),嘉澤 ( 1.46%),健策 ( 1.46%...",半導體業,10.78,"矽力*-KY ( 1.8%),京元電子 ( 1.42%),創意 ( 1.35%),力成 ( ...",電腦及週邊設備業,10.32,"仁寶 ( 1.88%),技嘉 ( 1.81%),微星 ( 1.61%),英業達 ( 1.58..."
2,0052,半導體業,79.01,"台積電 ( 65.39%),聯發科 ( 6.15%),聯電 ( 1.57%),日月光投控 (...",電腦及週邊設備業,8.48,"廣達 ( 1.78%),華碩 ( 1.28%),緯創 ( 0.77%),光寶科 ( 0.58...",其他電子業,6.05,"鴻海 ( 5.21%),可成 ( 0.38%),貿聯-KY ( 0.26%),鴻準 ( 0.2%)"
3,0053,半導體業,65.14,"台積電 ( 51.49%),聯發科 ( 5.59%),日月光投控 ( 1.4%),聯電 ( ...",電腦及週邊設備業,8.98,"廣達 ( 1.9%),華碩 ( 0.97%),緯穎 ( 0.7%),研華 ( 0.66%),...",電子零組件業,8.05,"台達電 ( 2.77%),川湖 ( 0.44%),嘉澤 ( 0.4%),台光電 ( 0.4%..."
4,0056,電腦及週邊設備業,22.18,"華碩 ( 3.45%),和碩 ( 2.49%),光寶科 ( 2.28%),廣達 ( 2.12...",半導體業,19.33,"聯詠 ( 4.08%),聯電 ( 3.73%),瑞昱 ( 3.49%),聯發科 ( 3.13...",金融保險,16.37,"中信金 ( 3.37%),華南金 ( 3.09%),兆豐金 ( 2.66%),上海商銀 ( ..."
5,0057,半導體業,60.60,"台積電 ( 50.78%),聯發科 ( 4.71%),聯電 ( 1.12%),日月光投控 (...",金融保險,11.70,"富邦金 ( 1.56%),中信金 ( 1.46%),國泰金 ( 1.3%),兆豐金 ( 1....",電腦及週邊設備業,6.16,"廣達 ( 1.35%),華碩 ( 0.96%),緯創 ( 0.58%),光寶科 ( 0.43..."
6,006203,半導體業,60.87,"台積電 ( 50.81%),聯發科 ( 4.82%),聯電 ( 1.13%),日月光投控 (...",金融保險,11.81,"富邦金 ( 1.58%),中信金 ( 1.47%),國泰金 ( 1.31%),兆豐金 ( 1...",電腦及週邊設備業,6.22,"廣達 ( 1.36%),華碩 ( 0.98%),緯創 ( 0.58%),光寶科 ( 0.43..."
7,006204,半導體業,44.36,"台積電 ( 34.07%),聯發科 ( 4.2%),聯電 ( 1.18%),日月光投控 ( ...",金融保險,14.03,"富邦金 ( 2.16%),國泰金 ( 1.7%),中信金 ( 1.46%),兆豐金 ( 1....",電腦及週邊設備業,7.10,"廣達 ( 1.64%),華碩 ( 0.85%),研華 ( 0.61%),緯穎 ( 0.54%..."
8,006208,半導體業,65.08,"台積電 ( 55.45%),聯發科 ( 5.19%),聯電 ( 1.32%),日月光投控 (...",金融保險,12.30,"富邦金 ( 1.75%),中信金 ( 1.63%),國泰金 ( 1.42%),兆豐金 ( 1...",電腦及週邊設備業,5.38,"廣達 ( 1.5%),華碩 ( 1.08%),緯創 ( 0.65%),光寶科 ( 0.49%..."
9,00690,半導體業,59.66,"台積電 ( 30.95%),聯發科 ( 16.19%),聯電 ( 4.03%),聯詠 ( 2...",電子零組件業,11.60,"台達電 ( 5.87%),國巨 ( 1.71%),台光電 ( 1.32%),嘉澤 ( 1.0...",金融保險,8.09,"富邦金 ( 6.51%),上海商銀 ( 1.58%)"


In [11]:

# export png
df_styled = df_result.style.background_gradient()
df_styled = df_result.style.set_properties(subset=['data','data_2','data_3'], **{'width': '600px'})
dfi.export(df_styled, "Result/mytable.png")

In [12]:
df_styled = (
    df_result.style
    .background_gradient()
    .set_properties(subset=['data', 'data_2', 'data_3'], **{'width': '600px'})
)

dfi.export(df_styled, "Result/mytable.png")